# Binary White Dwarf Timing Array (BWDTA) Analysis

[__1.__](#smbh) SMBH merger strain at MW barycenter

[__2.__](#bwd) BWD Preliminaries\
  a. Population Distribution of BWDs in the Milky Way (spatial)\
  b. Mass distribution \
  c. Frequency distribution\
  d. Peculiar velocity

[__3.__](#phase) Measuring the phase modulation using a BWD antenna

In [ ]:
import numpy as np
import scipy.constants as scc
from scipy.signal import butter, lfilter, freqs, filtfilt

import matplotlib.pyplot as plt, matplotlib as mpl
mpl.style.use(['fivethirtyeight'])

import matplotlib.colors as colors

In [ ]:
# Useful constants
M_sol = 1.989e30    # 1 solar mass in kg
H0 = 70             # Hubble's constant in km/s/Mpc

# Useful functions 

def kpcTom(k):
    ''' kiloparsec to metres'''
    return 3.086e19*k

def zTod(z):
    '''redshift to distance in parsecs'''
    return z * scc.c / H0 * 1e3

def fgwTop(f):
    '''freq in Hz to period in hours'''
    return 1/f / 3600

def Mchirp(M1, M2):
    ''' chirp mass from constituent masses'''
    return (M1 * M2)**(3/5) / (M1 + M2)**(1/5)

In [ ]:
# Quadrupole approximation strain estimate #

def h(r=1, f=1e-3, M=1):
    ''' Strain from the quadrupole formula
    r: distance in parsec
    M: solar masses
    f: GW frequency in Hz'''
    return 2 * (4 * np.pi)**2 * f**(2/3) * scc.G**(5/3) / scc.c**4 \
             * (M * M_sol)**(5/3) / (kpcTom(r))

<a name="smbh"> </a>

# 1. SMBH merger strain at MW barycenter

Assuming equal mass black holes with chirp mass range 100-10$^9 M_\odot$

Since we're interested in $10^{-10} \text{ to }10^{-6}$ Hz, the orbital period range considered is $10^6$ to $10^{10}$ s

In [ ]:
#def smbh_h(M,P,r=zTod(1)):
    #'''Calculates strain
    #M: in solar masses
    #P: in hours
    #r: in parsecs
    #'''
    # return 1e-21 * (M)**(5/3) * (P)**(-2/3) * (100/r)

In [ ]:
smbh_M_ = np.logspace(2,9,100)
smbh_f_ = np.logspace(-10,-6,100)


smbh_M, smbh_f = np.meshgrid(smbh_M_, smbh_f_)
smbh_h = h(r=zTod(1), f=smbh_f, M=smbh_M)

In [ ]:
levels = np.logspace(-29,-16,10)

fig, ax = plt.subplots(figsize=(10,8))
im = ax.pcolormesh(smbh_f, smbh_M, smbh_h, \
    norm=colors.LogNorm(vmin=smbh_h.min(), vmax=smbh_h.max()))
fig.colorbar(im, ax=ax)
cont = ax.contour(smbh_f, smbh_M, smbh_h , levels, linewidths=1, alpha=.9, colors='k', locator=plt.LogLocator())
ax.clabel(cont, levels, fmt='%1.0e',inline=True, use_clabeltext=True, rightside_up=1, fontsize=11)
ax.set_xlabel('GW freq [Hz]')
ax.set_ylabel('Chirp Mass [solar masses]')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_title('SMBH strain at z = 1');
#fig.savefig('SMBHStrainz1.pdf', bbox_inches='tight')

<a name="bwd"> </a>

## 2. BWD Preliminaries

The spatial, mass, and frequency distribution of the BWD population

In [ ]:
def dfdt(f=1e-3, M=1):
    '''Returns orbital evolution rate for BWD (Lamberts 2019 eq 5)
    f: Hz
    M: solar masses
    
    returns: Hz/year'''
    prefactor = (scc.G / scc.c**3 * M_sol)**(5/3) * 96/5 * np.pi**(8/3)
    return prefactor * M**(5/3) * f**(11/3) * (3600*24*365)

In [ ]:
mc_ = np.linspace(.3,2,50)
f_ = np.logspace(-4,-1,50)
mc,f = np.meshgrid(mc_,f_)
h_vals = h(f=f,M=mc)

In [ ]:
fig, ax = plt.subplots(2,1, sharex=True, figsize=(6,10))
ax[0].loglog(f_, dfdt(f=f_))

ax[0].set_ylabel('df/dt  [Hz/year]')

ax[1].loglog(f_, dfdt(f=f_)/f_)
ax[1].set_xlabel('GW frequency [Hz]')
ax[1].set_ylabel('dfdt/f  [1/year]')
fig.suptitle('Orbital frequency evolution of BWD');



### Mass distribution
Using the quadrupole formula (above) $h$ is plotted below as a function of chirp mass and GW frequency

In [ ]:
levels = np.logspace(-21,-17,10)
fig, ax = plt.subplots(figsize=(10,8))
im = ax.pcolormesh(f, mc, h_vals, \
    norm=colors.LogNorm(vmin=h_vals.min(), vmax=h_vals.max()))
fig.colorbar(im, ax=ax)
cont = ax.contour(f, mc, h_vals , levels, linewidths=.5,colors='k')
ax.clabel(cont, fmt='%1.0e',inline=1, fontsize=10)
ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel('Chirp Mass')
ax.set_xscale('log')
ax.set_title('Strain at 1 kPc');
#fig.savefig('Strain1kpc.pdf', bbox_inches='tight')

In [ ]:
1/24/3600

<a name="phase"> </a>

## 3. Phase Tracker

In order to be able to calculate the phase evolution of the modulation of GW signal measured at the earth from a single BWD, assuming we know the frequency of the GW signal from the BWD, $f_{GW}$, 



In [ ]:
sample_rate = 1e-3         # 1 Hz
max_time = 5 * 24*3600         # in days
order = 4                # filter order
safety = 1e-2

In [ ]:
fgw = 1e-3
fsig = 1e-7
hsig = 1e-16

In [ ]:
def butter_lowpass(data, cutoff, fs, order=5):
    nyq = 0.5 * fs
    normalCutoff = cutoff / nyq
    b, a = butter(order, normalCutoff, btype='low', analog=True)
    y = filtfilt(b, a, data)
    return y

def signal(t, fgw=fgw, fsig=fsig, hsig=hsig, phi=0):
    return np.cos(2*np.pi*fgw*t + hsig*np.cos(2*np.pi*fsig*t) - phi)

   

In [ ]:
t = np.linspace(0, max_time, int(max_time*sample_rate))
x_sig = signal(t)
x_noise = np.zeros_like(t)

x_data = x_sig + x_noise
y = butter_lowpass(x_data, cutoff, sample_rate, order)

Phase tracker:

In [ ]:
x_ref = signal(t, hsig=0, phi=np.pi/2)
prod = x_ref * x_sig

signal_data = butter_lowpass(prod, (1+safety)*fgw, sample_rate, order)

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
#ax.plot(t, x_sig, label='raw data')
ax.plot(t, signal_data, label='estimated signal')
#ax.plot(t, signal(t, fgw=fsig, hsig=0), label='true signal')
ax.legend()
#ax.plot(t, y)

## Other useful calculations

Chirp mass depending on individual masses:

In [ ]:
m_ = np.linspace(.17,1.33,20)
m1, m2 = np.meshgrid(m_,m_)
mc = Mchirp(m1,m2)

fig, ax = plt.subplots()
im = ax.pcolormesh(m1, m2, mc)
fig.colorbar(im, ax=ax)
cont = ax.contour(m1, m2, mc, linewidths=.5,colors='k')
ax.clabel(cont, fmt='%1.1f',inline=1, fontsize=10)
ax.set_xlabel('M1')
ax.set_ylabel('M2')
ax.set_title('Chirp mass')
#fig.tight_layout()